In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import shap
from sklearn.preprocessing import LabelEncoder


In [2]:
train_data = pd.read_csv('/Users/karolina/Desktop/Capstone/NSL_KDD_Train.csv')
test_data = pd.read_csv('/Users/karolina/Desktop/Capstone/NSL_KDD_Test.csv')

In [3]:
X_train = train_data.iloc[:, :-1]
y_train = train_data.iloc[:, -1]
X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]

# Align columns to keep only the common columns
common_columns = X_train.columns.intersection(X_test.columns)
X_train = X_train[common_columns]
X_test = X_test[common_columns]

# Label encode the target variable
y_train = y_train.astype(str)
y_test = y_test.astype(str)
all_labels = np.concatenate([y_train, y_test])

label_encoder = LabelEncoder()
label_encoder.fit(all_labels)
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)

categorical_columns = X_train.select_dtypes(include=['object']).columns

# Apply one-hot encoding to categorical columns
X_train = pd.get_dummies(X_train, columns=categorical_columns)
X_test = pd.get_dummies(X_test, columns=categorical_columns)

X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Ensure features are of type float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [4]:
nn_model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification
])
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the NN model
nn_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3937/3937 ━━━━━━━━━━━━━━━━━━━━ 2s 309us/step - accuracy: 0.0074 - loss: -169987152.0000
Epoch 2/10
3937/3937 ━━━━━━━━━━━━━━━━━━━━ 1s 305us/step - accuracy: 0.0076 - loss: -653793600.0000
Epoch 3/10
3937/3937 ━━━━━━━━━━━━━━━━━━━━ 1s 310us/step - accuracy: 0.0080 - loss: -399387968.0000
Epoch 4/10
3937/3937 ━━━━━━━━━━━━━━━━━━━━ 1s 314us/step - accuracy: 0.0073 - loss: -440694368.0000
Epoch 5/10
3937/3937 ━━━━━━━━━━━━━━━━━━━━ 1s 312us/step - accuracy: 0.0076 - loss: -528415584.0000
Epoch 6/10
3937/3937 ━━━━━━━━━━━━━━━━━━━━ 2s 392us/step - accuracy: 0.0077 - loss: -1412749312.0000
Epoch 7/10
3937/3937 ━━━━━━━━━━━━━━━━━━━━ 2s 629us/step - accuracy: 0.0074 - loss: -1385035648.0000
Epoch 8/10
3937/3937 ━━━━━━━━━━━━━━━━━━━━ 1s 354us/step - accuracy: 0.0074 - loss: -2245312256.0000
Epoch 9/10
3937/3937 ━━━━━━━━━━━━━━━━━━━━ 1s 315us/step - accuracy: 0.0078 - loss: -1950093952.0000
Epoch 10/10
3937/3937 ━━━━━━━━━━━━━━━━━━━━ 1s 322us/step - accuracy: 0.0073 - loss: -5783360000.0000


In [ ]:
explainer_nn = shap.KernelExplainer(nn_model.predict, shap.kmeans(X_train, 10))
shap_values_nn = explainer_nn.shap_values(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


  0%|          | 0/22543 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
652/652 ━━━━━━━━━━━━━━━━━━━━ 0s 198us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
652/652 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
654/654 ━━━━━━━━━━━━━━━━━━━━ 0s 365us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
653/653 ━━━━━━━━━━━━━━━━━━━━ 0s 408us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
653/653 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
653/653 ━━━━━━━━━━━━━━━━━━━━ 0s 395us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
653/653 ━━━━━━━━━━━━━━━━━━━━ 0s 370us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
653/653 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
654/654 ━━━━━━━━━━━━━━━━━━━━ 0s 408us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
652/652 ━━━━━━━━━━━━━━━━━━━━ 0s 423us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
653/653 ━━━━━━━━━━━━━━━━━━━━ 0s 418us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
652/652 ━━━━━━━━━━━━━━━━━━━━ 0s 355us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17

In [10]:
def perturb_inputs(X, feature_index, perturbation_value):
    X_perturbed = X.copy()
    X_perturbed.iloc[:, feature_index] = perturbation_value
    return X_perturbed

# Perturb feature index 2 (example feature index)
feature_to_perturb = 2
perturb_value = X_test.iloc[:, feature_to_perturb].mean() * 1.5
X_test_perturbed = perturb_inputs(X_test, feature_to_perturb, perturb_value)

# Analyze Label Flipping for NN
original_predictions_nn = nn_model.predict(X_test).flatten()
perturbed_predictions_nn = nn_model.predict(X_test_perturbed).flatten()

# Calculate flipping rate
flipping_nn = np.mean(original_predictions_nn != (perturbed_predictions_nn > 0.5))
print(f"Label flipping rate (NN): {flipping_nn}")

InvalidIndexError: (slice(None, None, None), 2)

In [ ]:
shap_values_nn_perturbed = explainer_nn.shap_values(X_test_perturbed)

# Visualize SHAP differences
shap.summary_plot(shap_values_nn, X_test, feature_names=X_test.columns)
shap.summary_plot(shap_values_nn_perturbed, X_test_perturbed, feature_names=X_test.columns)